In [87]:
import json
import numpy as np

IMAGE_NUM = 4883
CLASS_NUM = 10

In [88]:
json_file = open('./train.json')
infos = json.load(json_file)
json_file.close()

annotations = infos['annotations']

In [89]:
class_per_image = np.zeros((IMAGE_NUM, CLASS_NUM))
for info in annotations:
    class_per_image[info['image_id'], info['category_id']] += 1 

In [39]:
max_values = class_per_image.max(1)
max_class = class_per_image.argmax(1)
sorted_idx = max_values.argsort()[::-1]

In [48]:
K = 5
count_instance = np.zeros((K,10))
K_fold = [[]for i in range(K)]
for idx in sorted_idx:
    value = max_values[idx]
    class_idx = max_class[idx]
    min_fold = count_instance[:,class_idx].argmin()
    count_instance[min_fold] += class_per_image[idx]
    K_fold[min_fold].append(idx)

In [97]:
for i in range(K):
    print(f'{i} fold: {len(K_fold[i])}')

0 fold: 951
1 fold: 977
2 fold: 988
3 fold: 988
4 fold: 979


In [99]:
for i in range(K):
    print(f'{i} fold: {count_instance[i]}')

0 fold: [ 794. 1271.  181.  188.  197.  589.  253. 1036.   32.   94.]
1 fold: [ 793. 1270.  179.  187.  196.  588.  254. 1036.   33.   93.]
2 fold: [ 793. 1271.  179.  187.  197.  589.  252. 1035.   32.   94.]
3 fold: [ 793. 1270.  179.  187.  196.  588.  252. 1036.   31.   94.]
4 fold: [ 793. 1270.  179.  187.  196.  589.  252. 1035.   31.   93.]


## Save split

In [69]:
valid_fold_idx = 2

In [70]:
train_dict = dict()
valid_dict = dict()

train_dict['info'] = infos['info']
train_dict['licenses'] = infos['licenses']
train_dict['images'] = list()
train_dict['annotations'] = list()
train_dict['categories'] = infos['categories']
valid_dict['info'] = infos['info']
valid_dict['licenses'] = infos['licenses']
valid_dict['images'] = list()
valid_dict['annotations'] = list()
valid_dict['categories'] = infos['categories']

valid_inds = K_fold[valid_fold_idx]

In [73]:
for img_info in infos['images']:
    image_id = img_info['id']
    if image_id in valid_inds:
        valid_dict['images'].append(img_info)
    else:
        train_dict['images'].append(img_info)
        
for ann_info in infos['annotations']:
    image_id = ann_info['image_id']
    if image_id in valid_inds:
        valid_dict['annotations'].append(ann_info)
    else:
        train_dict['annotations'].append(ann_info)

In [75]:
with open("split_train.json", "w") as json_train:
    json.dump(train_dict, json_train)
with open("split_valid.json", "w") as json_valid:
    json.dump(valid_dict, json_valid)